In [ ]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

In [ ]:
#converting the model and code to desirable zip format supported by sagemaker inference
import tarfile

zipped_model_path = "model.tar.gz"

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add('model.pth')
    tar.add('code')

In [ ]:
#setting up exicution roles
session = sagemaker.Session()
role = get_execution_role()

In [ ]:
#uploading the model
bucket_name = 'ram-bucket'
s3_client = boto3.client('s3')
s3_client.upload_file('model.tar.gz', bucket_name, 'model.tar.gz')
s3_model_path = f's3://{bucket_name}/model.tar.gz'

In [ ]:
#Defining the PyTorchModel and set the entry_point as the name of the script that loads and runs your model
model = PyTorchModel(
    model_data=s3_model_path,
    role=role,
    framework_version='1.8.1',
    entry_point='inference.py',
    py_version="py3"
)

In [ ]:
#deploying the model to the spcified endpoint name
import time

endpoint_name = "object-detect" + time.strftime("%d-%m-%Y-%H-%M", time.gmtime())
predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name)

In [ ]:
#configuring endpoint to take image input and providing bounding boxes and it's lables as outputs
import base64
import json
image_path = 'Abyssinian.jpg'
with open(image_path, 'rb') as f:
    image_bytes = f.read()
image_base64 = base64.b64encode(image_bytes).decode('utf-8')

# Prepare the request payload
request_payload = {
    'input': image_base64
}
response = predictor.predict(json.dumps(request_payload))